In [1]:
import numpy as np
import math, json
from rdp import rdp

In [2]:
def create_animation(drawing, fps = 30, idx = 0, lw = 5): 
  
  seq_length = 0 
  
  xmax = 0 
  ymax = 0 
  
  xmin = math.inf
  ymin = math.inf
  
  #retreive min,max and the length of the drawing  
  for k in range(0, len(drawing)):
    x = drawing[k][0]
    y = drawing[k][1]

    seq_length += len(x)
    xmax = max([max(x), xmax]) 
    ymax = max([max(y), ymax]) 
    
    xmin = min([min(x), xmin]) 
    ymin = min([min(y), ymin]) 
    
  i = 0 
  j = 0
  
  # First set up the figure, the axis, and the plot element we want to animate
  fig = plt.figure()
  ax = plt.axes(xlim=(xmax+lw, xmin-lw), ylim=(ymax+lw, ymin-lw))
  ax.set_facecolor("white")
  line, = ax.plot([], [], lw=lw)

  #remove the axis 
  ax.grid = False
  ax.set_xticks([])
  ax.set_yticks([])
  
  # initialization function: plot the background of each frame
  def init():
      line.set_data([], [])
      return line, 

  # animation function.  This is called sequentially
  def animate(frame):    
    nonlocal i, j, line
    x = drawing[i][0]
    y = drawing[i][1]
    line.set_data(x[0:j], y[0:j])
    
    if j >= len(x):
      i +=1
      j = 0 
      line, = ax.plot([], [], lw=lw)
      
    else:
      j += 1
    return line,
  
  # call the animator.  blit=True means only re-draw the parts that have changed.
  anim = animation.FuncAnimation(fig, animate, init_func=init,
                                 frames= seq_length + len(drawing), blit=True)
  plt.close()
  
  # save the animation as an mp4.  
  anim.save(f'video.mp4', fps=fps, extra_args=['-vcodec', 'libx264'])

In [65]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [66]:
import glob
npy_files = glob.glob('server/data/*')
print(npy_files)

['server/data/Ryan_ريان.json']


In [68]:
drawing = json.load(open(npy_files[0]))

In [69]:
data = []
total_len_strokes = 0
for item in drawing:
    xs = []
    ys = []
    char = list(item.keys())[0]
    stroke = item[char]
    if len(stroke):
        print('processing ', char)
        print('current length of stroke ', len(stroke))
        post_stroke = rdp(stroke)
        print('post length of stroke ', len(post_stroke))
        total_len_strokes += len(post_stroke)
        for x, y in post_stroke:
            xs.append(500 - x)
            ys.append(y)
        data.append([xs, ys])

print(total_len_strokes)

processing  ر
current length of stroke  35
post length of stroke  35
processing  ى
current length of stroke  15
post length of stroke  15
processing  .
current length of stroke  1
post length of stroke  1
processing  .
current length of stroke  1
post length of stroke  1
processing  ا
current length of stroke  22
post length of stroke  22
processing  .
current length of stroke  1
post length of stroke  1
processing  ں
current length of stroke  58
post length of stroke  58
133


In [70]:
drawing

[{'ر': [[298.32586669921875, 235.19082641601562],
   [295.76849365234375, 246.0426025390625],
   [294.73175048828125, 248.94561767578125],
   [293.6258544921875, 252.40158081054688],
   [292.45086669921875, 255.09722900390625],
   [291.20672607421875, 258.2767333984375],
   [289.962646484375, 261.3179931640625],
   [288.5111083984375, 264.49749755859375],
   [286.85235595703125, 267.26226806640625],
   [284.84796142578125, 269.7506103515625],
   [283.11993408203125, 272.0315246582031],
   [281.1846923828125, 274.3816223144531],
   [278.83465576171875, 276.8699035644531],
   [276.76116943359375, 279.3582458496094],
   [274.4111328125, 281.77740478515625],
   [271.99200439453125, 284.12750244140625],
   [269.5037536621094, 285.9245910644531],
   [267.0155334472656, 288.1363830566406],
   [264.52728271484375, 290.07171630859375],
   [262.2464294433594, 291.3850402832031],
   [259.8272705078125, 292.7674255371094],
   [257.2699279785156, 294.3571472167969],
   [254.78167724609375, 295.3939

In [71]:
create_animation(data)

In [72]:
import io
import base64
video = io.open('video.mp4', 'r+b').read()

encoded = base64.b64encode(video)
HTML(data='''<video alt="video" autoplay>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))